# Fine-tuning de DistilBERT para detección de hate speech

En este notebook vamos a entrenar un modelo basado en transformers (DistilBERT)
para clasificar comentarios de YouTube como **tóxicos (1)** o **no tóxicos (0)**.

Objetivos:

- Cargar el dataset preprocesado (`text_basic` + `IsToxic`).
- Preparar los datos para HuggingFace (tokenizer + `Dataset`).
- Fine-tuning de `distilbert-base-uncased` para clasificación binaria.
- Evaluar el modelo en el conjunto de test (accuracy, precision, recall, F1, ROC-AUC).
- Guardar el modelo en `backend/models/distilbert_toxic_v1`.
- Guardar las métricas en `data/results/distilbert_toxic_v1.json`.


## 1. Configuración inicial y rutas del proyecto

En esta sección definimos las rutas relativas dentro del repositorio y hacemos
las importaciones básicas que necesitaremos más adelante.


In [14]:
import os
from pathlib import Path

# Ruta del repo (este notebook está en backend/notebooks/)
NOTEBOOK_DIR = Path.cwd()

# Detectar la raíz del proyecto de forma robusta
if NOTEBOOK_DIR.name == "notebooks":
    # Estamos en .../backend/notebooks
    BACKEND_DIR = NOTEBOOK_DIR.parent
    ROOT_DIR = BACKEND_DIR.parent
elif NOTEBOOK_DIR.name == "backend":
    # Estamos en .../backend
    BACKEND_DIR = NOTEBOOK_DIR
    ROOT_DIR = BACKEND_DIR.parent
else:
    # Asumimos que estamos en la raíz del proyecto
    ROOT_DIR = NOTEBOOK_DIR
    BACKEND_DIR = ROOT_DIR / "backend"

DATA_DIR = ROOT_DIR / "data"
PREPROC_DIR = DATA_DIR / "preprocessing_data"
RESULTS_DIR = DATA_DIR / "results"
MODELS_DIR = BACKEND_DIR / "models"

# Ficheros concretos
CSV_PATH = PREPROC_DIR / "youtoxic_english_1000_clean.csv"
METRICS_JSON_PATH = RESULTS_DIR / "distilbert_toxic_v1.json"
DISTILBERT_MODEL_DIR = MODELS_DIR / "distilbert_toxic_v1"

print("Notebook dir:", NOTEBOOK_DIR)
print("Root dir:", ROOT_DIR)
print("Backend dir:", BACKEND_DIR)
print("CSV path:", CSV_PATH)
print("CSV exists?", CSV_PATH.exists())
print("Metrics JSON:", METRICS_JSON_PATH)
print("Model dir:", DISTILBERT_MODEL_DIR)

Notebook dir: c:\Users\yeder\Documents\Factoria F5 Bootcamp IA\Proyecto_X_NLP_G4\backend\notebooks
Root dir: c:\Users\yeder\Documents\Factoria F5 Bootcamp IA\Proyecto_X_NLP_G4
Backend dir: c:\Users\yeder\Documents\Factoria F5 Bootcamp IA\Proyecto_X_NLP_G4\backend
CSV path: c:\Users\yeder\Documents\Factoria F5 Bootcamp IA\Proyecto_X_NLP_G4\data\preprocessing_data\youtoxic_english_1000_clean.csv
CSV exists? True
Metrics JSON: c:\Users\yeder\Documents\Factoria F5 Bootcamp IA\Proyecto_X_NLP_G4\data\results\distilbert_toxic_v1.json
Model dir: c:\Users\yeder\Documents\Factoria F5 Bootcamp IA\Proyecto_X_NLP_G4\backend\models\distilbert_toxic_v1


## 2. Instalación de dependencias

Instalamos las librerías necesarias para trabajar con transformers y los
datasets de HuggingFace. Esta celda solo es necesario ejecutarla la primera vez
(en el entorno del proyecto).


In [2]:
%pip install -q "transformers>=4.40.0" "datasets>=2.19.0" "evaluate" "accelerate" "scikit-learn"


Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -q "tf_keras"

Note: you may need to restart the kernel to use updated packages.


## 3. Imports principales

Importamos las librerías de HuggingFace, sklearn y utilidades varias.


In [23]:
import numpy as np
import pandas as pd
import torch

from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
import evaluate
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
)


## 4. Carga del dataset preprocesado

Cargamos el CSV que generamos en el notebook de preprocesado.  
Suponemos que tiene al menos estas columnas:

- `text_basic`: texto preparado para modelos modernos.
- `IsToxic`: etiqueta binaria (0 = no tóxico, 1 = tóxico).


In [24]:
df = pd.read_csv(CSV_PATH)

print(df.head())
print("\nColumnas:", df.columns.tolist())
print("\nDistribución de IsToxic:")
print(df["IsToxic"].value_counts(normalize=True))


              CommentId      VideoId  \
0  Ugg2KwwX0V8-aXgCoAEC  04kJtp6pVXI   
1  Ugg2s5AzSPioEXgCoAEC  04kJtp6pVXI   
2  Ugg3dWTOxryFfHgCoAEC  04kJtp6pVXI   
3  Ugg7Gd006w1MPngCoAEC  04kJtp6pVXI   
4  Ugg8FfTbbNF8IngCoAEC  04kJtp6pVXI   

                                                Text  IsToxic  IsAbusive  \
0  If only people would just take a step back and...    False      False   
1  Law enforcement is not trained to shoot to app...     True       True   
2  \r\nDont you reckon them 'black lives matter' ...     True       True   
3  There are a very large number of people who do...    False      False   
4  The Arab dude is absolutely right, he should h...    False      False   

   IsThreat  IsProvocative  IsObscene  IsHatespeech  IsRacist  \
0     False          False      False         False     False   
1     False          False      False         False     False   
2     False          False       True         False     False   
3     False          False      False     

### 4.1 Limpieza ligera y renombrado de la columna label

HuggingFace `Trainer` espera normalmente una columna `labels`.  
Renombramos `IsToxic` → `label` y nos aseguramos de que sea un entero.


In [25]:
# Eliminamos filas con textos o etiquetas nulas por seguridad
df = df.dropna(subset=["text_basic", "IsToxic"]).reset_index(drop=True)

# Renombrar la columna de target a 'label'
df = df.rename(columns={"IsToxic": "label"})
df["label"] = df["label"].astype(int)

df[["text_basic", "label"]].head()


,text_basic,label
0,If only people would just take a step back and...,0
1,Law enforcement is not trained to shoot to app...,1
2,Dont you reckon them 'black lives matter' bann...,1
3,There are a very large number of people who do...,0
4,"The Arab dude is absolutely right, he should h...",0


## 5. División en train y test

Dividimos el dataset en entrenamiento (80%) y test (20%), estratificando por la
etiqueta para mantener la proporción de tóxicos y no tóxicos.


In [26]:
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

len(train_df), len(test_df)


(797, 200)

## 6. Tokenizer y conversión a `datasets.Dataset`

Usamos el modelo **`distilbert-base-uncased`** y su tokenizer oficial.  
Luego convertimos los `DataFrame` de pandas a objetos `Dataset` y aplicamos
la tokenización con padding y truncado.


In [27]:
MODEL_NAME = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_batch(batch):
    return tokenizer(
        batch["text_basic"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

# Creamos Dataset a partir de pandas
train_ds = Dataset.from_pandas(train_df[["text_basic", "label"]])
test_ds = Dataset.from_pandas(test_df[["text_basic", "label"]])

# Aplicamos tokenización
train_encoded = train_ds.map(tokenize_batch, batched=True)
test_encoded = test_ds.map(tokenize_batch, batched=True)

# Renombramos 'label' -> 'labels' y preparamos tensores para PyTorch
train_encoded = train_encoded.rename_column("label", "labels")
test_encoded = test_encoded.rename_column("label", "labels")

train_encoded.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)
test_encoded.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)

train_encoded, test_encoded


Map:   0%|          | 0/797 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

(Dataset({
     features: ['text_basic', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
     num_rows: 797
 }),
 Dataset({
     features: ['text_basic', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
     num_rows: 200
 }))

## 7. Modelo DistilBERT para clasificación binaria

Cargamos `AutoModelForSequenceClassification` con `num_labels=2` para que
DistilBERT aprenda a distinguir entre comentarios tóxicos y no tóxicos.


In [28]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 8. Métricas de evaluación

Usamos las métricas de HuggingFace `evaluate` para:

- `accuracy`
- `precision`
- `recall`
- `f1`

Después, calcularemos también `ROC-AUC` manualmente con `sklearn`.


In [29]:
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    acc = accuracy_metric.compute(predictions=preds, references=labels)["accuracy"]
    prec = precision_metric.compute(predictions=preds, references=labels, average="binary")["precision"]
    rec = recall_metric.compute(predictions=preds, references=labels, average="binary")["recall"]
    f1 = f1_metric.compute(predictions=preds, references=labels, average="binary")["f1"]

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
    }


## 9. Parámetros de entrenamiento

Configuramos los `TrainingArguments` para el fine-tuning:

- 3 épocas
- batch size 16
- aprendizaje 5e-5
- evaluación al final de cada época
- guardar el mejor modelo según F1


In [31]:
training_args = TrainingArguments(
    output_dir=str(ROOT_DIR / "distilbert_training_outputs"),
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,

    # 🔧 Cambios para compatibilidad
    eval_strategy="epoch",
    save_strategy="epoch",

    logging_steps=20,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)


## 10. Entrenamiento del modelo

Creamos el objeto `Trainer` pasando:

- modelo
- argumentos de entrenamiento
- datasets codificados
- tokenizer
- función de métricas

Luego lanzamos el entrenamiento.


In [32]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encoded,
    eval_dataset=test_encoded,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

train_result = trainer.train()
train_result


C:\Users\yeder\AppData\Local\Temp\ipykernel_17516\3246262577.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\yeder\Documents\Factoria F5 Bootcamp IA\Proyecto_X_NLP_G4\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.557900,0.503095,0.755000,0.712871,0.782609,0.746114
2,0.312400,0.481691,0.775000,0.790123,0.695652,0.739884
3,0.171300,0.531706,0.800000,0.782609,0.782609,0.782609


c:\Users\yeder\Documents\Factoria F5 Bootcamp IA\Proyecto_X_NLP_G4\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\yeder\Documents\Factoria F5 Bootcamp IA\Proyecto_X_NLP_G4\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=150, training_loss=0.3737753423055013, metrics={'train_runtime': 507.0915, 'train_samples_per_second': 4.715, 'train_steps_per_second': 0.296, 'total_flos': 79182387546624.0, 'train_loss': 0.3737753423055013, 'epoch': 3.0})

## 11. Evaluación final en el conjunto de test

Usamos el `Trainer` para obtener predicciones en el conjunto de test y
calculamos métricas detalladas:

- Accuracy
- Precision
- Recall
- F1
- ROC-AUC
- Matriz de confusión (TN, FP, FN, TP)


In [33]:
# Predicciones crudas
pred_output = trainer.predict(test_encoded)

logits = pred_output.predictions
labels = pred_output.label_ids

# Predicción final (clase 0/1)
preds = np.argmax(logits, axis=-1)

# Probabilidad de la clase positiva para ROC-AUC
probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()
pos_probs = probs[:, 1]

accuracy = accuracy_score(labels, preds)
precision = precision_score(labels, preds)
recall = recall_score(labels, preds)
f1 = f1_score(labels, preds)
roc_auc = roc_auc_score(labels, pos_probs)

cm = confusion_matrix(labels, preds)
tn, fp, fn, tp = cm.ravel()

metrics_dict = {
    "accuracy": float(accuracy),
    "precision": float(precision),
    "recall": float(recall),
    "f1": float(f1),
    "roc_auc": float(roc_auc),
    "tn": int(tn),
    "fp": int(fp),
    "fn": int(fn),
    "tp": int(tp),
}

metrics_dict


c:\Users\yeder\Documents\Factoria F5 Bootcamp IA\Proyecto_X_NLP_G4\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'accuracy': 0.8,
 'precision': 0.782608695652174,
 'recall': 0.782608695652174,
 'f1': 0.782608695652174,
 'roc_auc': 0.8649355877616747,
 'tn': 88,
 'fp': 20,
 'fn': 20,
 'tp': 72}

## 12. Guardar el modelo fine-tuneado

Guardamos el modelo y el tokenizer en la carpeta de modelos del backend:

`backend/models/distilbert_toxic_v1`


In [34]:
DISTILBERT_MODEL_DIR.mkdir(parents=True, exist_ok=True)

model.save_pretrained(DISTILBERT_MODEL_DIR)
tokenizer.save_pretrained(DISTILBERT_MODEL_DIR)

print(f"Modelo guardado en: {DISTILBERT_MODEL_DIR}")


Modelo guardado en: c:\Users\yeder\Documents\Factoria F5 Bootcamp IA\Proyecto_X_NLP_G4\backend\models\distilbert_toxic_v1


## 13. Exportar métricas a JSON para la app

Generamos un JSON con la información del modelo y las métricas para poder:

- compararlo en `comparison_models.ipynb`
- visualizarlo en el frontend (pestaña Resultados)


In [35]:
import json
from datetime import datetime

METRICS_JSON_PATH.parent.mkdir(parents=True, exist_ok=True)

output_json = {
    "model_name": "distilbert_toxic_v1",
    "task": "binary_classification",
    "target_label": "IsToxic",
    "data": {
        "n_samples": int(len(df)),
        "n_train": int(len(train_df)),
        "n_test": int(len(test_df)),
        "train_size": len(train_df) / len(df),
        "test_size": len(test_df) / len(df),
        "random_state": 42,
    },
    "metrics": {
        "accuracy": metrics_dict["accuracy"],
        "precision": metrics_dict["precision"],
        "recall": metrics_dict["recall"],
        "f1": metrics_dict["f1"],
        "roc_auc": metrics_dict["roc_auc"],
    },
    "confusion_matrix": {
        "tn": metrics_dict["tn"],
        "fp": metrics_dict["fp"],
        "fn": metrics_dict["fn"],
        "tp": metrics_dict["tp"],
    },
    "timestamp": datetime.utcnow().isoformat(),
    "notes": "DistilBERT fine-tuned on text_basic for hate speech detection",
}

with open(METRICS_JSON_PATH, "w", encoding="utf-8") as f:
    json.dump(output_json, f, indent=2)

output_json


{'model_name': 'distilbert_toxic_v1',
 'task': 'binary_classification',
 'target_label': 'IsToxic',
 'data': {'n_samples': 997,
  'n_train': 797,
  'n_test': 200,
  'train_size': 0.7993981945837513,
  'test_size': 0.20060180541624875,
  'random_state': 42},
 'metrics': {'accuracy': 0.8,
  'precision': 0.782608695652174,
  'recall': 0.782608695652174,
  'f1': 0.782608695652174,
  'roc_auc': 0.8649355877616747},
 'confusion_matrix': {'tn': 88, 'fp': 20, 'fn': 20, 'tp': 72},
 'timestamp': '2025-12-10T13:43:13.161030',
 'notes': 'DistilBERT fine-tuned on text_basic for hate speech detection'}

## 14. Función de inferencia rápida

Definimos una pequeña función para probar el modelo con un texto suelto.
Esto viene muy bien para hacer pruebas desde el notebook y también como base
para el endpoint del backend.


In [36]:
id2label = {0: "NO TOXIC (0)", 1: "TOXIC (1)"}

def predict_text(text: str):
    encoded = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt",
    )

    with torch.no_grad():
        outputs = model(**encoded)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1).numpy()[0]
        pred = int(np.argmax(probs))

    return {
        "text": text,
        "predicted_label": pred,
        "label_name": id2label[pred],
        "score": float(probs[pred]),
    }

test_example = "You are disgusting, go away!"
predict_text(test_example)


{'text': 'You are disgusting, go away!',
 'predicted_label': 1,
 'label_name': 'TOXIC (1)',
 'score': 0.9710241556167603}

## 15. ¿Qué es un transformer? Explicación intuitiva

Imagina una frase:

> "This video is sick!"

La palabra **"sick"** puede significar _enfermo_ o _muy bueno_.  
Para entenderlo bien, necesitamos mirar el **contexto completo**.

### Modelos clásicos (RNN / LSTM)

- Leen la frase palabra a palabra: `This → video → is → sick!`
- Intentan recordar el contexto en una "memoria" interna.
- Si la frase es larga, esa memoria se degrada y se olvidan cosas.

### Transformers: atención a todo

Un transformer hace algo distinto:  
para cada palabra, mira a **todas las demás** y decide qué tan importantes son.

Para la palabra `sick`, puede aprender:

- que `video` es muy relevante,
- que `This` o `is` importan menos,
- y que el signo `!` puede indicar emoción.

Esto se llama **self-attention (auto-atención)**.

Matemáticamente, se calculan pesos del tipo:

- attention(`sick`, `video`) = 0.91  
- attention(`sick`, `this`) = 0.13

Con esos pesos, el modelo mezcla la información de todas las palabras y
construye una representación muy rica de la frase completa.

### ¿Por qué es tan potente?

- No lee el texto solo de izquierda a derecha, sino que ve la frase **entera de golpe**.
- Puede capturar relaciones entre palabras lejanas.
- Funciona genial para:
  - detección de toxicidad
  - análisis de sentimiento
  - traducción
  - chatbots (como este proyecto 💬)

### ¿Y DistilBERT qué es?

- Es una versión **reducida y más rápida** de BERT.
- Conserva la mayor parte de su rendimiento (~95% de la calidad).
- Es ideal para proyectos de producción o demos donde queremos:
  - buen rendimiento,
  - pero sin un modelo gigantesco.

En nuestro caso:

- DistilBERT ya viene **pre-entrenado** en muchísimo texto.
- Solo le hemos hecho _fine-tuning_ para que aprenda una tarea muy concreta:
  decidir si un comentario de YouTube es **tóxico (1)** o **no tóxico (0)**.
